##MNIST

In [0]:
def mnist_data():
  #transforms.Compose makes multiple transformations at once.
  #transforms.Normalize((mean1,mean2. . . mean n), (std1, std2. . . std n)) normalized n dimensional data with respective means and standard deviations
  compose=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
  out_dir='./dataset' #Directory where mnist needs to be downladed
  #download the mnist dataset in output directory 'out_dir' and applies transformations 'compose'
  # here the dataset is returned as a tensor
  return datasets.MNIST(root=out_dir,train=True,transform=compose, download=True)

data=mnist_data()

data_loader=torch.utils.data.DataLoader(data, batch_size=100,shuffle=True)
num_batches=len(data_loader)

##CIFAR 10

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

train_size=int(train_split*len(dataset))
val_size=len(dataset)-train_size
trainset, valset=torch.utils.data.random_split(dataset,[train_size,val_size])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print("Size of train set",len(trainloader))
print("Size of val set",len(valloader))
print("Size of test set",len(testloader))



# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
print("Size of images",images.size())


##Custom Data loader template 

##Pair wise image mapping Data loader
Images with same label in two folder train A and train B

##Pair wise image mapping Data loader with one OCR image and a text generated image
Image with label in folder train. Pair image generated using label